In [ ]:
# Step 1: Mount Google Drive to access our files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 2: Install the libraries we are going to use
# OpenCV is for image and video processing.
# Ultralytics gives us access to the YOLO model to detect players.
!pip install opencv-python ultralytics

In [ ]:
# Step 3: Load and test our video
import cv2
from google.colab.patches import cv2_imshow # We use this in Colab instead of cv2.imshow

# IMPORTANT: Paste here the path to your video that you copied in the previous step.
# Make sure the path is inside quotation marks.
video_path = "/content/drive/My Drive/Proyecto_Futbol_IA/mi_partido.mp4"

# We try to open the video
cap = cv2.VideoCapture(video_path)

# We check if the video opened correctly
if not cap.isOpened():
  print("Error: No se pudo abrir el vídeo. Revisa la ruta del archivo.")
else:
  print("¡Éxito! El vídeo se ha cargado correctamente.")
  # We read the first frame of the video
  ret, frame = cap.read()
  if ret:
    print("Mostrando el primer fotograma del vídeo:")
    # We display the frame in Colab
    cv2_imshow(frame)
  else:
    print("Error: No se pudo leer ningún fotograma del vídeo.")
  # We release the video object
  cap.release()